In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
from lsst.sims.maf.metrics import BaseMetric

import os
import sys
sys.path.append("../../LSST_OpSim/Scripts_NBs/")
from opsimUtils import *

In [ ]:
#We will use the same slicer and constraint for each metric. Since there are no warnings about cache
#and minimum nside, we will use 64 to go faster.
NSIDE=64
slicer = slicers.HealpixSlicer(nside=NSIDE)
constraint = 'visitExposureTime > 11'

In [ ]:
metric = metrics.CountMetric(col='observationStartMJD', metricName='nvisitsLong')

In [ ]:
mb = metricBundles.MetricBundle(metric, slicer, constraint)

In [ ]:
bundleDict = {'VisitCounts':mb}

In [ ]:
your_username = "rjassef"
folder_mafoutput = "WFDfootprint_{0:d}".format(NSIDE)
outDir = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(your_username,folder_mafoutput)
if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [ ]:
metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

FBS_versions = ["1.5","1.6","1.7"]
for FBS_version in FBS_versions:
    dbDir = '/home/idies/workspace/lsst_cadence/FBS_{}/'.format(FBS_version)
    opSimDbs, resultDbs = connect_dbs(dbDir, outDir)
    dbRuns = show_opsims(dbDir)
    for run in dbRuns:
        mb.setRunName(run)
        metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                        opSimDbs[run], metricDataPath, resultDbs[run])
        metricGroup.runAll()